In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentence-transformers faiss-cpu transformers torch numpy pandas


In [3]:

import os
import re
import numpy as np
import faiss
import pandas as pd

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [4]:
import os

DATA_DIR = "/content/drive/MyDrive/Semester 2/NLP/Mini RAG/data_txt"

os.listdir(DATA_DIR)


['berita1.txt',
 'berita2.txt',
 'berita3.txt',
 'berita4.txt',
 'berita5.txt',
 'berita6.txt',
 'berita7.txt',
 'berita8.txt',
 'berita9.txt',
 'berita10.txt',
 'berita11.txt',
 'berita12.txt',
 'berita13.txt',
 'berita14.txt',
 'berita15.txt',
 'berita16.txt',
 'berita17.txt',
 'berita18.txt',
 'berita19.txt',
 'berita20.txt',
 'berita21.txt',
 'berita22.txt',
 'berita23.txt',
 'berita24.txt',
 'berita25.txt',
 'berita32.txt',
 'berita33.txt',
 'berita35.txt',
 'berita40.txt',
 'berita41.txt',
 'berita42.txt',
 'berita43.txt',
 'berita45.txt',
 'berita44.txt',
 'berita46.txt',
 'berita47.txt',
 'berita48.txt',
 'berita49.txt',
 'berita50.txt',
 'berita26.txt',
 'berita27.txt',
 'berita31.txt',
 'berita36.txt',
 'berita37.txt',
 'berita38.txt',
 'berita39.txt',
 'berita34.txt',
 'berita30.txt',
 'berita29.txt',
 'berita28.txt']

In [5]:
documents = []
sources = []
for file in os.listdir(DATA_DIR):
    if file.endswith(".txt"):
        with open(os.path.join(DATA_DIR, file), "r", encoding="utf-8") as f:
            text =" ".join(f.read().split())
            documents.append(text)
            sources.append(file)

print("Jumlah dokumen:", len(documents))

Jumlah dokumen: 50


query & expected answer

In [6]:

queries = [
    "Kenapa banjir sumatera bisa terjadi?",
    "Daerah mana saja yang terdampak parah banjir sumatera?",
    "Tanggal berapa banjir terjadi",
    "Bantuan apa saja telah diberikan?",
    "Infrastruktur apa yang rusak akibat banjir?"
]

expected_answers = [
    "Curah hujan ekstrem, kerusakan hutan, dan siklon tropis",
    "Aceh, Sumatera Utara, dan Sumatera Barat",
    "Dimulai dari 24 November 2025",
    "Obat-obatan, sembako dan pakaian",
    "Jalan, Rumah, Puskesmas dan banyak bangunan penting lainnya"
]

chunking (150 token, overlap 30)

In [7]:
import re
def sentence_chunking(text, max_chars=400):
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ""
    for s in sentences:
        # Perbaikan: Tambahkan spasi antar kalimat agar tidak menempel
        if len(current) + len(s) + 1 <= max_chars:
            current += (" " + s if current else s)
        else:
            chunks.append(current.strip())
            current = s
    if current.strip():
        chunks.append(current.strip())
    return chunks

In [8]:
chunks = []

for doc_id, doc in enumerate(documents):
    for chunk in sentence_chunking(doc):
        chunks.append({
            "text": chunk,
            "doc_id": doc_id
        })

print("Total chunks:", len(chunks))


Total chunks: 705


Embedding (BGE-M3)

In [9]:
#!pip install sentence-transformers faiss-cpu


In [10]:
embedding_models = {
    "bge-m3": SentenceTransformer("BAAI/bge-m3"),
    "miniLM-multi": SentenceTransformer(
        "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
}

chunk_texts = [c["text"] for c in chunks]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
chunk_embeddings_all = {}
faiss_indexes = {}

for name, model in embedding_models.items():
    print(f"Embedding with {name}...")
    emb = model.encode(
        chunk_texts,
        normalize_embeddings=True,
        show_progress_bar=True
    )

    index = faiss.IndexFlatIP(emb.shape[1])
    index.add(emb)

    chunk_embeddings_all[name] = emb
    faiss_indexes[name] = index


Embedding with bge-m3...


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Embedding with miniLM-multi...


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Retrieval Function

In [12]:
def retrieve_chunks(query, model_name="bge-m3", k=3):
    model = embedding_models[model_name]
    index = faiss_indexes[model_name]

    q_emb = model.encode([query], normalize_embeddings=True)
    scores, indices = index.search(q_emb, k)

    return [chunks[i]["text"] for i in indices[0]]


MMR

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr_rerank(query_emb, doc_embs, docs, k=3, lambda_param=0.7):
    selected, selected_idx = [], []
    sim_query = cosine_similarity(query_emb, doc_embs)[0]

    for _ in range(k):
        if not selected_idx:
            idx = np.argmax(sim_query)
        else:
            sim_selected = cosine_similarity(doc_embs[selected_idx], doc_embs)
            diversity = np.max(sim_selected, axis=0)
            mmr_score = lambda_param * sim_query - (1 - lambda_param) * diversity
            idx = np.argmax(mmr_score)

        selected_idx.append(idx)
        selected.append(docs[idx])

    return selected


In [14]:
def retrieve_chunks_mmr(query, model_name="bge-m3", top_n=10, k=3):
    model = embedding_models[model_name]
    index = faiss_indexes[model_name]

    q_emb = model.encode([query], normalize_embeddings=True)
    scores, indices = index.search(q_emb, top_n)

    docs = [chunks[i]["text"] for i in indices[0]]
    doc_embs = chunk_embeddings_all[model_name][indices[0]]

    return mmr_rerank(q_emb, doc_embs, docs, k=k)


Load LLM (Flan-T5)

In [15]:
llm_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(llm_name)
model = AutoModelForSeq2SeqLM.from_pretrained(llm_name)

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256
)


Device set to use cuda:0


Non-RAG Answer

In [16]:
def non_rag_answer(query):
  prompt = f"Jawab pertanyaan berikut secara singkat:\n{query}"
  return generator(prompt)[0]["generated_text"]


RAG Answer

In [17]:
def rag_answer(query, model_name="bge-m3", k=3):
    retrieved = retrieve_chunks_mmr(
        query,
        model_name=model_name,
        k=k
    )

    context = "\n".join(retrieved)

    prompt = f"""
Answer the following question using ONLY the context.

Context:
{context}

Question:
{query}

Answer:
"""

    answer = generator(prompt, do_sample=False)[0]["generated_text"]
    return answer.strip(), retrieved


In [18]:
test_ans, test_ctx = rag_answer("Apa penyebab banjir?", k=3)
print(test_ans)


Sumatra 2025?Penyebabnya adalah hujan ekstrem berkepanjangan yang bertemu dengan kerusakan lingkungan di hulu dan lemahnya daya tampung sungai. Wilayah mana paling terdampak?Aceh, Sumatera Utara, dan Sumatera Barat menjadi tiga provinsi dengan dampak paling luas. Fenomena ini bersifat anomali dan sangat terjadi dalam intensitas sebesar itu. Kerusakan lingkungan berperan sangat besar dalam memperparah dampak banjir.


Perbandingan

In [19]:
results_bge = []

for i, q in enumerate(queries):
    non_rag = non_rag_answer(q)
    rag, retrieved = rag_answer(
        q,
        model_name="bge-m3",
        k=3
    )

    results_bge.append({
        "Question": q,
        "Expected Answer": expected_answers[i],
        "RAG Answer (BGE-M3)": rag,
        "Non-RAG Answer": non_rag
    })

df_bge = pd.DataFrame(results_bge)

display(df_bge)


Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,Question,Expected Answer,RAG Answer (BGE-M3),Non-RAG Answer
0,Kenapa banjir sumatera bisa terjadi?,"Curah hujan ekstrem, kerusakan hutan, dan sikl...","Utara, dan Sumatera Barat menjadi tiga provins...",Pertanyaan pertanyaan pertanyaan sebagai bahwa...
1,Daerah mana saja yang terdampak parah banjir s...,"Aceh, Sumatera Utara, dan Sumatera Barat","Aceh, Sumatera Utara, dan Sumatera Barat menja...",Pertanyaan pertanyaan sekarang tidak adalah ti...
2,Tanggal berapa banjir terjadi,Dimulai dari 24 November 2025,Sumatera diketahui mengalami bencana banjir be...,Jawab pertanyaan pertanyaan secara yang terjad...
3,Bantuan apa saja telah diberikan?,"Obat-obatan, sembako dan pakaian","obat-obatan, pakaian layak pakai, serta dukung...",Pertanyaan pertanyaan pertanyaan sejak: Bantua...
4,Infrastruktur apa yang rusak akibat banjir?,"Jalan, Rumah, Puskesmas dan banyak bangunan pe...",arus keluar masuk kenaraan roda dua dan empat ...,The main topic is: What is Russian infrastruct...


In [20]:
results_mini = []

for i, q in enumerate(queries):
    non_rag = non_rag_answer(q)
    rag, retrieved = rag_answer(
        q,
        model_name="miniLM-multi",
        k=3
    )


    results_mini.append({
        "Question": q,
        "Expected Answer": expected_answers[i],
        "RAG Answer (MiniLM)": rag,
        "Non-RAG Answer": non_rag
    })

df_mini = pd.DataFrame(results_mini)
display(df_mini)


,Question,Expected Answer,RAG Answer (MiniLM),Non-RAG Answer
0,Kenapa banjir sumatera bisa terjadi?,"Curah hujan ekstrem, kerusakan hutan, dan sikl...",untuk membantuan hujan tinggi dan siklon tropi...,Pertanyaan pertanyaan pertanyaan sebagai bahwa...
1,Daerah mana saja yang terdampak parah banjir s...,"Aceh, Sumatera Utara, dan Sumatera Barat","Aceh, Sumatera Utara, dan Sumatera Barat",Pertanyaan pertanyaan sekarang tidak adalah ti...
2,Tanggal berapa banjir terjadi,Dimulai dari 24 November 2025,Muksalmina. Banjir sebenarnya bukan hal baru b...,Jawab pertanyaan pertanyaan secara yang terjad...
3,Bantuan apa saja telah diberikan?,"Obat-obatan, sembako dan pakaian","dalam bentuk sembako, obat-obatan, pakaian lay...",Pertanyaan pertanyaan pertanyaan sejak: Bantua...
4,Infrastruktur apa yang rusak akibat banjir?,"Jalan, Rumah, Puskesmas dan banyak bangunan pe...",membawa material berat seperti batu dan kayu g...,The main topic is: What is Russian infrastruct...


In [21]:
comparison_df = df_bge.merge(
    df_mini,
    on=["Question", "Expected Answer", "Non-RAG Answer"]
)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
display(comparison_df)

,Question,Expected Answer,RAG Answer (BGE-M3),Non-RAG Answer,RAG Answer (MiniLM)
0,Kenapa banjir sumatera bisa terjadi?,"Curah hujan ekstrem, kerusakan hutan, dan siklon tropis","Utara, dan Sumatera Barat menjadi tiga provinsi dengan dampak paling luas",Pertanyaan pertanyaan pertanyaan sebagai bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa bahwa,untuk membantuan hujan tinggi dan siklon tropis. Judul: Mengapa Penangan Banjir Sumatra Lambat
1,Daerah mana saja yang terdampak parah banjir sumatera?,"Aceh, Sumatera Utara, dan Sumatera Barat","Aceh, Sumatera Utara, dan Sumatera Barat menjadi tiga provinsi dengan dampak paling luas. Lalu di Sumatera Utara (Sumut) ada sebanyak 306 titik, terdiri atas 180 titik terdampak banjir dan 126 titik terdampak longsor. Kemudian di Sumatera Barat (Sumbar) ada sebanyak 630 titik, terdiri atas 427 akibat banjir dan 203 akibat longsor.",Pertanyaan pertanyaan sekarang tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah tidak adalah,"Aceh, Sumatera Utara, dan Sumatera Barat"
2,Tanggal berapa banjir terjadi,Dimulai dari 24 November 2025,"Sumatera diketahui mengalami bencana banjir beberapa kali di tahun 2025. Salah satu yang terparah terjadi pada bulan November ketika banjir bandang dan longsor melanda beberapa wilayah, terutama di Sumatera Utara dan Sumatera Barat. Bencana ini dipicu oleh curah hujan ekstrem yang terjadi sejak Senin, 24 November 2025.",Jawab pertanyaan pertanyaan secara yang terjadi: Tanggal berapa banjir terjadi,"Muksalmina. Banjir sebenarnya bukan hal baru bagi Muksalmina. Namun, derasnya debit air pada akhir November lalu datang jauh lebih cepat from biasanya, hingga memaksanya bertahan selama dua hari dua malam di atas tanggul irigasi dekat rumahnya. Tak ada yang tersisa from rumah tempat tinggalnya selain pondasi. Dalam kurun waktu empat hari, banjir dan longsor melanda tiga provinsi: Aceh, Sumatera Utara, dan Sumatera Barat. Total wilayah terdampak mencapai ratusan kecamatan, mencakup daerah perbukitan, dataran rendah, hingga pesisir."
3,Bantuan apa saja telah diberikan?,"Obat-obatan, sembako dan pakaian","obat-obatan, pakaian layak pakai, serta dukungan tenaga dan peralatan untuk membantu proses pembersihan wilayah terdampak banjir, bekerja sama dengan pemerintah daerah dan instansi terkait",Pertanyaan pertanyaan pertanyaan sejak: Bantuan what else has been done?,"dalam bentuk sembako, obat-obatan, pakaian layak pakai, serta dukungan tenaga dan peralatan untuk membantuan proses pembersihan wilayah terdampak banjir, bekerja sama dengan pemerintah daerah dan instansi terkait"
4,Infrastruktur apa yang rusak akibat banjir?,"Jalan, Rumah, Puskesmas dan banyak bangunan penting lainnya",arus keluar masuk kenaraan roda dua dan empat lumpuh karena badan jalan dan jembatan putus dihantam banjir,The main topic is: What is Russian infrastructure?,"membawa material berat seperti batu dan kayu gelondongan, merusak banyak rumah warga serta berbagai fasilitas umum"


Gradio

In [22]:
#!pip install -U gradio


In [23]:
def gradio_qa(query, mode, model_name, k):
    if mode == "Non-RAG":
        answer = non_rag_answer(query)
        return (
            answer,
            "— (No retrieval used)"
        )

    answer, retrieved = rag_answer(
        query=query,
        model_name=model_name,
        k=int(k)
    )

    context = "\n\n".join(
        [f"[Chunk {i+1}]\n{chunk}" for i, chunk in enumerate(retrieved)]
    )

    return (
        answer,
        context
    )


In [24]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## Sistem Mini RAG pada Banjir Sumatera")
    gr.Markdown(
        "Ajukan Pertanyaan mu! "
    )

    query_input = gr.Textbox(
        label="Pertanyaan",
        placeholder="Contoh: Apa penyebab utama banjir di Sumatera?",
        lines=2
    )

    with gr.Row():
        mode_input = gr.Radio(
            choices=["RAG", "Non-RAG"],
            value="RAG",
            label="Mode Jawaban"
        )

        model_input = gr.Dropdown(
            choices=list(embedding_models.keys()),
            value="bge-m3",
            label="Embedding Model"
        )

        k_input = gr.Dropdown(
            choices=[1, 3, 5],
            value=3,
            label="Jumlah Retrieved Chunks (k)"
        )

    submit_btn = gr.Button("Tanya")

    answer_output = gr.Textbox(
        label="Jawaban",
        lines=4
    )

    context_output = gr.Textbox(
        label="Retrieved Context (RAG)",
        lines=10
    )


    submit_btn.click(
        fn=gradio_qa,
        inputs=[query_input, mode_input, model_input, k_input],
        outputs=[
            answer_output,
            context_output

        ]
    )



In [25]:
demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://98aff0c6f41cf1058a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://98aff0c6f41cf1058a.gradio.live


In [ ]:
demo.launch(
    share=True,
    debug=True
)


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f5370c6456d5c7be6e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
